<a href="https://colab.research.google.com/github/marcosferreiracabral/projeto_pyspark_aviacao/blob/main/pyspark_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalação das dependências:

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Fazendo download
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# Descompactando os arquivos
!tar xf spark-3.3.0-bin-hadoop3.tgz

# instalando a findspark
!pip install -q findspark

!pip install -q pyspark


Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state info

In [3]:
# Configuração das variáveis de ambiente:

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

# Tornar o pyspark "importável":

import findspark

findspark.init('spark-3.3.0-bin-hadoop3')


In [4]:
# Iniciar uma sessão local:

import pyspark
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()


In [5]:
# Importação das bibliotecas para o notebook:

from pyspark.sql.functions import col, expr, when


# Airports Dataset:

In [6]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1cpygs-tfjUqAYqbBAkt3zcVP1h4J_3W_/view?usp=sharing

# Carregar dados do Airports:

df_airports = spark.read.csv("./airports.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_airports.printSchema()


view?usp=sharing.22     [ <=>                ]  66.57K  --.-KB/s    in 0.01s   
root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



In [7]:
# Tipo do dado:

type(df_airports)

pyspark.sql.dataframe.DataFrame

In [8]:
# Lendo o dataframe e impimindo as colunas e linhas:

df_airports.show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

Perguntas<br>
Considere o dataset airports.csv para realizar as seguintes tarefas:

In [9]:
# Lendo o dataframe e impimindo o cabeçalho:

df_airports.head(10) # 10 primeiras linhas.


[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.431912, lon=-74.3915611, alt=523, tz=-5, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.0744722, lon=-81.4277778, alt=11, tz=-4, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.3712222, lon=-82.1734167, alt=1593, tz=-4, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.4673056, lon=-84.5067778, alt=730, tz=-5, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.8835647, lon=-76.7812318, alt=492, tz=-5, dst='A'),
 Row(faa='0P2', name='Shoestring Aviation Airfield', lat=39.7948244, lon=-76.6471914, alt=1000, tz=-5, dst='U'),
 Row(faa='0S9', name='Jefferson 

In [10]:
# Contagem das linhas da coluna faa:

df_airports.select(col('faa')).count()


1397

In [11]:
# Tipo da coluna faa:

df_airports.select(col('faa')).dtypes


[('faa', 'string')]

In [12]:
# Retorna os dados em um Array:
df_airports.select(col('faa')).collect()


[Row(faa='04G'),
 Row(faa='06A'),
 Row(faa='06C'),
 Row(faa='06N'),
 Row(faa='09J'),
 Row(faa='0A9'),
 Row(faa='0G6'),
 Row(faa='0G7'),
 Row(faa='0P2'),
 Row(faa='0S9'),
 Row(faa='0W3'),
 Row(faa='10C'),
 Row(faa='17G'),
 Row(faa='19A'),
 Row(faa='1A3'),
 Row(faa='1B9'),
 Row(faa='1C9'),
 Row(faa='1CS'),
 Row(faa='1G3'),
 Row(faa='1OH'),
 Row(faa='1RL'),
 Row(faa='24C'),
 Row(faa='24J'),
 Row(faa='25D'),
 Row(faa='29D'),
 Row(faa='2A0'),
 Row(faa='2G2'),
 Row(faa='2G9'),
 Row(faa='2J9'),
 Row(faa='369'),
 Row(faa='36U'),
 Row(faa='38W'),
 Row(faa='3D2'),
 Row(faa='3G3'),
 Row(faa='3G4'),
 Row(faa='3J1'),
 Row(faa='3W2'),
 Row(faa='40J'),
 Row(faa='41N'),
 Row(faa='47A'),
 Row(faa='49A'),
 Row(faa='49X'),
 Row(faa='4A4'),
 Row(faa='4A7'),
 Row(faa='4A9'),
 Row(faa='4B8'),
 Row(faa='4G0'),
 Row(faa='4G2'),
 Row(faa='4G4'),
 Row(faa='4I7'),
 Row(faa='4U9'),
 Row(faa='52A'),
 Row(faa='54J'),
 Row(faa='55J'),
 Row(faa='57C'),
 Row(faa='60J'),
 Row(faa='6A2'),
 Row(faa='6K8'),
 Row(faa='6S0'

In [13]:
# Retorna um novo DataFramecontendo as linhas distintas neste DataFrame:

df_airports.select(col('faa')).distinct().count()


1396

In [14]:
# Retorna um novo DataFrame contendo as linhas distintas neste DataFrame:

df_airports.select(col('faa')).explain()


== Physical Plan ==
FileScan csv [faa#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/airports.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<faa:string>




In [15]:
# Substitui os valores nulos:

df_airports.fillna('').show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [16]:
# Retorna um novo DataFrameomitindo linhas com valores nulos:

df_airports.dropna().show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [17]:
# Função inclui contagem, média, stddev, min e max. Se nenhuma coluna for fornecida, esta função calcula estatísticas para todas as colunas numéricas ou de string:

df_airports.describe(['faa']).show()


+-------+------------------+
|summary|               faa|
+-------+------------------+
|  count|              1397|
|   mean|             141.0|
| stddev|197.46392075515973|
|    min|               04G|
|    max|               ZYP|
+-------+------------------+



1 - Crie a coluna qa_faa e aponte inconsistências da coluna faa de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formator de 3-5 caracteres alfanuméricos.<br>
Coluna retorna m ou f

In [18]:
df_airports_qa_faa = df_airports.withColumn("qa_faa", when(df_airports.faa == '', 'M')
                                 .when(df_airports.faa.rlike('a-zA-Z0-9'),'F')
                                 .when(df_airports.faa.between(3, 5),'F')
                                 .otherwise(df_airports.faa))
df_airports_qa_faa.show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_faa|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|   04G|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|   06A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|   06C|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|   06N|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|   09J|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|   0A9|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|   0G6|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|   0G7|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|   0P2|
|0S9|Jefferson C

2 - Crie a coluna qa_name e aponte inconsistências da coluna name de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.

In [19]:
df_airports_qa_name = df_airports.withColumn("qa_name", when(df_airports.name == '', 'M')
                                 .otherwise(df_airports.name))
df_airports_qa_name.show()


+---+--------------------+----------------+-----------------+----+---+---+--------------------+
|faa|                name|             lat|              lon| alt| tz|dst|             qa_name|
+---+--------------------+----------------+-----------------+----+---+---+--------------------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|   Lansdowne Airport|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|Moton Field Munic...|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A| Schaumburg Regional|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|     Randall Airport|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|Jekyll Island Air...|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|Elizabethton Muni...|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|Williams County A...|
|0G7|Finger Lakes Regi...|      42.88356

3 - Crie a coluna qa_lat e aponte inconsistências da coluna lat de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-180, 180].<br>
A : Indica que o valor é alfanumérico.

In [20]:
df_airports_qa_lat = df_airports.withColumn("qa_lat", when(df_airports.lat == '', 'M')
                                 .when(df_airports.lat.between(-180, 180),'I')
                                 .when(df_airports.lat.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_airports.lat))
df_airports_qa_lat.show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lat|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|     I|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|     I|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|     I|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|     I|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|     I|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|     I|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|     I|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|     I|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|     I|
|0S9|Jefferson C

4 - Crie a coluna qa_lon e aponte inconsistências da coluna lon de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-180, 180].<br>
A : Indica que o valor é alfanumérico.

In [21]:
df_airports_qa_lon = df_airports.withColumn("qa_lon", when(df_airports.lon == '', 'M')
                                 .when(df_airports.lon.between(-180, 180),'I')
                                 .when(df_airports.lon.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_airports.lon))
df_airports_qa_lon.show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lon|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|     I|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|     I|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|     I|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|     I|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|     I|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|     I|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|     I|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|     I|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|     I|
|0S9|Jefferson C

5 - Crie a coluna qa_alt e aponte inconsistências da coluna alt de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [0,+∞).<br>
A : Indica que o valor é alfanumérico.<br>

In [22]:
df_airports_qa_alt = df_airports.withColumn("qa_alt", when(df_airports.alt == '', 'M')
                                 .when(df_airports.alt.between(0, +0),'I')
                                 .when(df_airports.alt.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_airports.alt))
df_airports_qa_alt.show()



+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_alt|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|  1044|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|   264|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|   801|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|   523|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|    11|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|  1593|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|   730|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|   492|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|  1000|
|0S9|Jefferson C

6 - Crie a coluna qa_tz e aponte inconsistências da coluna tz de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [-11, +14].<br>
A : Indica que o valor é alfanumérico.

In [23]:
df_airports_qa_tz = df_airports.withColumn("qa_tz", when(df_airports.tz == '', 'M')
                                 .when(df_airports.tz.between(-11, +14),'I')
                                 .when(df_airports.tz.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_airports.tz))
df_airports_qa_tz.show()


+---+--------------------+----------------+-----------------+----+---+---+-----+
|faa|                name|             lat|              lon| alt| tz|dst|qa_tz|
+---+--------------------+----------------+-----------------+----+---+---+-----+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|    I|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|    I|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|    I|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|    I|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|    I|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|    I|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|    I|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|    I|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|    I|
|0S9|Jefferson County ...|  

7 - Crie a coluna qa_dst e aponte inconsistências da coluna dst de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma das categorias esperadas: E, A, S, O, Z, N, U<br>
N : Indica que o valor é numérico.

In [24]:
df_airports_qa_dst = df_airports.withColumn("qa_dst", when(df_airports.dst == '', 'M')
                                 .when(df_airports.dst == 'E', 'C')
                                 .when(df_airports.dst == 'A', 'C')
                                 .when(df_airports.dst == 'S', 'C')
                                 .when(df_airports.dst == 'O', 'C')
                                 .when(df_airports.dst == 'Z', 'C')
                                 .when(df_airports.dst == 'N', 'C')
                                 .when(df_airports.dst == 'U', 'C')
                                 .when(df_airports.dst.rlike('a-zA-Z0-9'),'N')                                 
                                 .otherwise(df_airports.dst))
df_airports_qa_dst.show()

+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_dst|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|     C|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|     C|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|     C|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|     C|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|     C|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|     C|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|     C|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|     C|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|     C|
|0S9|Jefferson C

# Planes Dataset:

In [25]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/18Ru1UuLHv9d6nPBWj5-T_ikyJAHhaDI2/view?usp=sharing

# Carregar dados do Planes:

df_planes = spark.read.csv("./planes.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_planes.printSchema()


view?usp=sharing.23     [ <=>                ]  66.58K  --.-KB/s    in 0.01s   
root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: string (nullable = true)
 |-- engine: string (nullable = true)



In [26]:
df_planes.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

In [27]:
df_planes.fillna('').show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

Perguntas<br>
Considere o dataset planes.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem exatamente 5 caracteres.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

In [28]:
df_planes_qa_tailnum = df_planes.withColumn('qa_tailnum', when(df_planes.tailnum == '', 'M')
                                 .when(df_planes.tailnum.rlike('N1234Z'),'A')
                                 .when(df_planes.tailnum.rlike('N123AZ'),'A')
                                 .when(df_planes.tailnum == 'I', 'FN')
                                 .when(df_planes.tailnum == '0', 'FE')
                                 .when(df_planes.tailnum == 'Z', 'FE')
                                 .when(df_planes.tailnum.rlike('a-zA-Z0-9'),'N')
                                 .otherwise(df_planes.tailnum))
df_planes_qa_tailnum.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N102UW|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N103US|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N104UW|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N105UW|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N107US|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|    N108UW|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

2 - Crie a coluna qa_year e aponte inconsistências da coluna year de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1950,+∞).

In [29]:
df_planes_qa_year = df_planes.withColumn('qa_year', when(df_planes. year == '', 'M')
                                 .when(df_planes.year.between(1950, 2022),'I')                             
                                 .otherwise(df_planes.year))
df_planes_qa_year.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      I|


3 - Crie a coluna qa_type e aponte inconsistências da coluna type de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Fixed wing multi engine<br>
Fixed wing single engine<br>
Rotorcraft

In [30]:
df_planes_qa_type = df_planes.withColumn('qa_type', when(df_planes.type == '', 'M')
                                 .when(df_planes.type == 'Fixed wing multi engine', 'C')
                                 .when(df_planes.type == 'Fixed wing single engine', 'C')
                                 .when(df_planes.type == 'Rotorcraft', 'C')
                                 .otherwise(df_planes.type))
df_planes_qa_type.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_type|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      C|


4 - Crie a coluna qa_manufacturer e aponte inconsistências da coluna manufacturer de acordo com
as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
AIRBUS<br>
BOEING<br>
BOMBARDIER<br>
CESSNA<br>
EMBRAER<br>
SIKORSKY<br>
CANADAIR<br>
PIPER<br>
MCDONNELL DOUGLAS<br>
CIRRUS<br>
BELL<br>
KILDALL GARY<br>
LAMBERT RICHARD<br>
BARKER JACK<br>
ROBINSON HELICOPTER<br>
GULFSTREAM<br>
MARZ BARRY<br>

In [31]:
df_planes_qa_manufacturer = df_planes.withColumn('qa_manufacturer', when(df_planes.manufacturer == '', 'M')
                                 .when(df_planes.manufacturer == 'AIRBUS', 'C')
                                 .when(df_planes.manufacturer == 'BOEING', 'C')
                                 .when(df_planes.manufacturer == 'BOMBARDIER', 'C')
                                 .when(df_planes.manufacturer == 'CESSNA', 'C')
                                 .when(df_planes.manufacturer == 'EMBRAER', 'C')
                                 .when(df_planes.manufacturer == 'SIKORSKY', 'C')
                                 .when(df_planes.manufacturer == 'CANADAIR', 'C')
                                 .when(df_planes.manufacturer == 'PIPER', 'C')
                                 .when(df_planes.manufacturer == 'MCDONNELL DOUGLAS', 'C')
                                 .when(df_planes.manufacturer == 'CIRRUS', 'C')
                                 .when(df_planes.manufacturer == 'BELL', 'C')
                                 .when(df_planes.manufacturer == 'KILDALL GARY', 'C')
                                 .when(df_planes.manufacturer == 'LAMBERT RICHARD', 'C')
                                 .when(df_planes.manufacturer == 'BARKER JACK', 'C')
                                 .when(df_planes.manufacturer == 'ROBINSON HELICOPTER', 'C')
                                 .when(df_planes.manufacturer == 'GULFSTREAM', 'C')
                                 .when(df_planes.manufacturer == 'MARZ BARRY', 'C')
                                 .otherwise(df_planes.manufacturer))
df_planes_qa_manufacturer.show()



+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine| qa_manufacturer|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|AIRBUS INDUSTRIE|
| N109UW|1999|Fixed

5 - Crie a coluna qa_model e aponte inconsistências da coluna model de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado<br>
Modelos AIRBUS devem começar com "A"<br>
Modelos BOEING devem começar com "7"<br>
Modelos BOMBARDIER e CANADAIR devem começar com "CL"<br>
Modelos MCDONNELL DOUGLAS devem começar com "MD" ou "DC"

In [32]:
df_planes_qa_model = df_planes.withColumn('qa_model', when(df_planes.model == '', 'M')
                                 .when(df_planes.model.rlike('A'),'F')
                                 .when(df_planes.model.rlike('7'),'F')
                                 .when(df_planes.model.rlike('CL'),'F')
                                 .when(df_planes.model.rlike('MD'),'F')
                                 .when(df_planes.model.rlike('DC'),'F')
                                 .otherwise(df_planes.model))
df_planes_qa_model.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_model|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       F|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

6 - Crie a coluna qa_engines e aponte inconsistências da coluna engines de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1, 4].<br>
A : Indica que o valor é alfanumérico.

In [33]:
df_planes_qa_engines = df_planes.withColumn("qa_engines", when(df_planes.engines == '', 'M')
                                 .when(df_planes.engines.between(1, 4),'I')
                                 .when(df_planes.engines.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_planes.engines))

df_planes_qa_engines.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engines|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|         I|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

7 - Crie a coluna qa_seats e aponte inconsistências da coluna seats de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br><br>
I : Indica que o valor excede o intervalo [2, 500].<br>
A : Indica que o valor é alfanumérico.

In [34]:
df_planes_qa_seats = df_planes.withColumn("qa_seats", when(df_planes.seats.between(2, 500),'I')
                                 .when(df_planes.seats.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_planes.seats))

df_planes_qa_seats.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_seats|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|       I|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

8 - Crie a coluna qa_speed e aponte inconsistências da coluna speed de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 150].<br>
A : Indica que o valor é alfanumérico.<br>

In [35]:
df_planes_qa_speed = df_planes.withColumn("qa_speed", when(df_planes.speed == '', 'M')
                                 .when(df_planes.speed.between(50, 150),'I')
                                 .when(df_planes.speed.rlike('a-zA-Z0-9'),'A')                                 
                                 .otherwise(df_planes.speed))

df_planes_qa_speed.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_speed|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|      NA|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|

9 - Crie a coluna qa_engine e aponte inconsistências da coluna engine de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Turbo-fan<br>
Turbo-jet<br>
Turbo-prop<br>
Turbo-shaft<br>
4 Cycle

In [36]:
df_planes_qa_engine = df_planes.withColumn('qa_engine', when(df_planes.engine == '', 'M')
                                 .when(df_planes.engine.rlike('Turbo-fan'),'C')
                                 .when(df_planes.engine.rlike('Turbo-jet'),'C')
                                 .when(df_planes.engine.rlike('Turbo-prop'),'C')
                                 .when(df_planes.engine.rlike('Turbo-shaft'),'C')
                                 .when(df_planes.engine.rlike('4 Cycle'),'C')
                                 .otherwise(df_planes.engine))
df_planes_qa_engine.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|        C|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|T

# Flights Dataset:

In [37]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1ggZ_dBMvAkILRh-YrozqsvsXc2XYUYeG/view?usp=sharing

# Carregar dados do Flights:

df_flights = spark.read.csv("./flights.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_flights.printSchema()


view?usp=sharing.24     [ <=>                ]  66.56K  --.-KB/s    in 0.03s   
root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [38]:
df_flights.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [39]:
df_flights.count()

10000

In [40]:
df_flights.fillna('').show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [41]:
df_flights.fillna('').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Perguntas<br>
Considere o dataset flights.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_year_month_day e aponte inconsistências das colunas year , month , day de
acordo com as regras abaixo.<br>
MY : Indica que está com dado faltante no ano.<br>
MM : Indica que está com dado faltante no mes.<br>
MD : Indica que está com dado faltante no dia.<br>
IY : Indica que o valor excede o intervalo [1950,+∞] no ano.<br>
IM : Indica que o valor excede o intervalo [1, 12] no mês.<br>
ID : Indica que o valor excede o intervalo [1, 31] no dia. No mês de Fevereiro o intervalo é [1, 29].

In [42]:
df_flights_qa_year_month_day = df_flights.withColumn("qa_year_month_day", when(df_flights.year == '','MY')
                                 .when(df_flights.month == '','MM')
                                 .when(df_flights.month == '','MD')
                                 .when(df_flights.year.between(1950, 2022),'IY')
                                 .when(df_flights.month.between(1, 12),'IM')
                                 .when(df_flights.day.between(1, 31),'ID')
                                 .when(df_flights.day.between(1, 29),'ID')
                                 .otherwise(df_flights.year))

df_flights_qa_year_month_day.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_year_month_day|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               IY|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               IY|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|               IY|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|               IY|
|2014|    3|  9|     754|       -1

2 - Crie a coluna qa_hour_minute e aponte inconsistencias das colunas hour e minute de acordo
com as regras abaixo.<br>
MH : Indica que está com dado faltante na hora.<br>
MM : Indica que está com dado faltante no minuto.<br>
IH : Indica que o valor excede o intervalo na hora [0, 24].<br>
IM : Indica que o valor excede o intervalo no minuto [0, 59].<br>

In [43]:
df_flights_qa_hour_minute = df_flights.withColumn("qa_hour_minute", when(df_flights.hour == '','MH')
                                 .when(df_flights.minute == '','MM')
                                 .when(df_flights.hour.between(0, 24),'IY')
                                 .when(df_flights.minute.between(0, 59),'IM')
                                 .otherwise(df_flights.hour))
df_flights_qa_hour_minute.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_hour_minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|            IY|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|            IY|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|            IY|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|            IY|
|2014|    3|  9|     754|       -1|    1015|        1| 

3 - Crie a coluna qa_dep_arr_time e aponte inconsistências da coluna dep_time e arr_time de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_time .<br>
MA : Indica que está com dado faltante no arr_time .<br>
FD : Indica que não respeita o formato esperado (HHMM ou HMM) no dep_time .<br>
FA : Indica que não respeita o formato esperado (HHMM ou HMM) no arr_time .<br>

In [44]:
df_flights_qa_dep_arr_time = df_flights.withColumn('qa_dep_arr_time', when(df_flights.dep_time == '', 'MD')
                                 .when(df_flights.dep_time.rlike('HHMM'),'FD')
                                 .when(df_flights.dep_time.rlike('HMM'),'FD')
                                 .when(df_flights.arr_time == '', 'MA')
                                 .when(df_flights.arr_time.rlike('HHMM'),'FA')
                                 .when(df_flights.arr_time.rlike('HMM'),'FA')
                                 .otherwise(df_flights.dep_time))
df_flights_qa_dep_arr_time.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_time|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|            658|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|           1040|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|           1443|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|           1705|
|2014|    3|  9|     754|       -1|    1015|    

4 - Crie a coluna qa_dep_arr_delay e aponte inconsistências da coluna dep_delay e arr_delay de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_delay .<br>
MA : Indica que está com dado faltante no arr_delay .<br>

In [45]:
df_flights_qa_dep_arr_delay = df_flights.withColumn('qa_dep_arr_delay', when(df_flights.dep_delay == '', 'MD')

                                 .when(df_flights.arr_delay == '', 'MA')

                                 .otherwise(df_flights.dep_delay))
df_flights_qa_dep_arr_delay.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|              -7|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               5|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              -2|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|              45|
|2014|    3|  9|     754|       -1|    10

5 - Crie a coluna qa_carrier e aponte inconsistências da coluna carrier de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (2 caracteres alfanuméricos).<br>

In [46]:
df_flights_qa_carrier = df_flights.withColumn("qa_year_month_day", when(df_flights.carrier == '','M')
                                 .when(df_flights.carrier.between(0, 2),'F')
                                 .otherwise(df_flights.carrier))

df_flights_qa_carrier.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_year_month_day|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               VX|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               AS|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|               VX|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|               WN|
|2014|    3|  9|     754|       -1

6 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem o número de caracteres esperado.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

In [47]:
df_flights_qa_tailnum = df_flights.withColumn('qa_tailnum', when(df_flights.tailnum == '', 'M')
                                 .when(df_flights.carrier.between(0, 6),'S')
                                 .when(df_flights.tailnum.rlike('N1234Z'),'F')
                                 .when(df_flights.tailnum.rlike('N123AZ'),'F')
                                 .when(df_flights.tailnum.rlike('N'),'FN')
                                 .when(df_flights.tailnum.rlike('I'),'FE')
                                 .when(df_flights.tailnum.rlike('O'),'FE')
                                 .when(df_flights.tailnum.rlike('0'),'FE')
                                 .otherwise(df_flights.tailnum))
df_flights_qa_tailnum.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_tailnum|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|        FN|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|        FN|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        FN|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|        FN|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA

7 - Crie a coluna qa_flight e aponte inconsistências da coluna flight de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (4 caracteres numéricos).<br>

In [48]:
df_flights_qa_flight = df_flights.withColumn('qa_flight', when(df_flights.flight == '', 'M')
                                 .when(df_flights.flight.between(0, 4),'F')
                                 .otherwise(df_flights.flight))
df_flights_qa_flight.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_flight|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|     1780|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|      851|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|      755|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|      344|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR| 

8 - Crie a coluna qa_origin_dest e aponte inconsistências da coluna origin , dest de acordo com
as regras abaixo.<br>
MO : Indica que está com dado faltante no origin .<br>
MD : Indica que está com dado faltante no dest .<br>
FO : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no origin .<br>
FD : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no dest .<br>

In [49]:
df_flights_qa_origin_dest = df_flights.withColumn('qa_origin_dest', when(df_flights.origin == '', 'MO')
                                 .when(df_flights.origin.between(0, 3),'FO')
                                 .when(df_flights.dest == '', 'MO')
                                 .when(df_flights.dest.between(0, 3),'FD')
                                 .otherwise(df_flights.origin))
df_flights_qa_origin_dest.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_origin_dest|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|           SEA|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|           SEA|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|           SEA|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|           PDX|
|2014|    3|  9|     754|       -1|    1015|        1| 

9 - Crie a coluna qa_air_time e aponte inconsistencias da coluna air_time de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [20, 500].<br>

In [50]:
df_flights_qa_air_time = df_flights.withColumn(' qa_air_time', when(df_flights.air_time == '', 'M')
                                 .when(df_flights.air_time.between(20, 500),'I')
                                 .otherwise(df_flights.air_time))
df_flights_qa_air_time.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute| qa_air_time|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|           I|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|           I|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|           I|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|           I|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS

10 - Crie a coluna qa_distance e aponte inconsistências da coluna distance de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 3000].<br>

In [51]:
df_flights_qa_distance = df_flights.withColumn('qa_distance', when(df_flights.distance == '', 'M')
                                 .when(df_flights.distance.between(50, 3000),'I')
                                 .otherwise(df_flights.distance))
df_flights_qa_distance.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_distance|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|          I|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|          I|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|          I|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|          I|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522

11 - Crie a coluna qa_distance_airtime e aponte inconsistências entre as colunas distance e
air_time de acordo com as regras abaixo.<br>
M : Indica que está com distance ou air_time faltante.<br>
TL : Indica que a viagem é longa de acordo com a condição:<br>
air_time >= distance × 0.1 + 30.<br>
TS : Indica que a viagem é curta de acordo com a condição:<br>
air_time <= distance × 0.1 + 10.<br>
TR : Indica que a viagem é normal caso as duas anteriores não sejam verdade.<br>

In [52]:
# o símbolo ~ representa a condição de negação:

df_flights_qa_distance_airtime = df_flights.withColumn('qa_distance_airtime', when(df_flights.flight == '', 'M')
                                 .when(df_flights.air_time == '', 'TL')
                                 .when(df_flights.air_time >= df_flights.distance * 0.1 + 30, 'TL')
                                 .when(df_flights.air_time >= df_flights.distance * 0.1 + 10, 'TS')
                                 .when(~(df_flights.air_time >= df_flights.distance * 0.1 + 30)
                                 == (df_flights.air_time >= df_flights.distance * 0.1 + 10), 'TR')
                                 .otherwise(df_flights.air_time))
df_flights_qa_distance_airtime.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|qa_distance_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|                 TL|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|                 TL|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|                 TL|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|                 TS|
|2014|    3|  9|    